In [ ]:
# run these 3 then res runtime

!pip3 install --upgrade --force-reinstall protobuf
!pip3 install  --upgrade --force-reinstall grpcio-tools
!pip install --upgrade --force-reinstall pymilvus


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load Vectors from a pickle file

In [ ]:
import pickle

with open('vectors.pkl', 'rb') as f:
  data = pickle.load(f)

In [ ]:
data_list = []
for d in data:
    data_list.append(d)

In [ ]:
data_ids = [data_list[i]["id"] for i in range(len(data_list))]
data_vecs = [data_list[i]["vector"] for i in range(len(data_list))]

Create Connection

In [ ]:
from pymilvus import connections, Collection, CollectionSchema, FieldSchema, DataType

connections.connect("default", uri="http://localhost:19530")


Create Collection

In [ ]:
TABLE_NAME = "<TABLE_NAME>"
field_name = "<VECTOR_FIELD_NAME>"

pk = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False)
field = FieldSchema(name=field_name, dtype=DataType.FLOAT_VECTOR, dim=768)
schema = CollectionSchema(fields=[pk,field], description="vectors collection")
collection = Collection(name=TABLE_NAME, schema=schema)

Bulk load part

In [ ]:
mr_list = []
for i in range(0, len(data_list), 10000):
  connections.connect("default", uri="http://localhost:19530")
  collection = Collection(name=TABLE_NAME)

  mr = collection.insert([data_ids[i:i+10000], data_vecs[i:i+10000]], timeout=60)
  mr_item = {"step":i, "err_count": mr.err_count, "succ_count": mr.succ_count}
  mr_list.append(mr_item)
  connections.remove_connection("default")

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [ ]:
mr_list # get data from mr object.

[{'step': 0, 'err_count': 0, 'succ_count': 10000},
 {'step': 10000, 'err_count': 0, 'succ_count': 10000},
 {'step': 20000, 'err_count': 0, 'succ_count': 10000},
 {'step': 30000, 'err_count': 0, 'succ_count': 10000},
 {'step': 40000, 'err_count': 0, 'succ_count': 10000},
 {'step': 50000, 'err_count': 0, 'succ_count': 10000},
 {'step': 60000, 'err_count': 0, 'succ_count': 10000},
 {'step': 70000, 'err_count': 0, 'succ_count': 10000},
 {'step': 80000, 'err_count': 0, 'succ_count': 10000},
 {'step': 90000, 'err_count': 0, 'succ_count': 5003}]

Build an Index on Vectors


---



---


Vector indexes are an organizational unit of metadata used to accelerate vector similarity search. Without the index built on vectors, Milvus will perform a brute-force search by default.


In [ ]:
connections.connect("default", uri="http://localhost:19530")
collection = Collection(name="address")

index_param = {
        "metric_type":"IP",
         "index_type":"IVF_SQ8",
        # "index_type": "AUTOINDEX",
        "params":{"nlist":1024}
    }
collection.release(timeout=60)
collection.create_index(field_name=field_name, index_params=index_param)
collection.load()


In [ ]:
collection.num_entities # entity count in the collection

In [ ]:
connections.remove_connection("default")

In [ ]:
# DROPS COLLECTION

#from pymilvus import utility
#utility.drop_collection("address")